# 1. Create population

#### 1.1 Import customer list from CSV

In [79]:
import pandas as pd

# path to customers list
csv_file_path = 'customers.csv'

# create pandas dataframe with all customers from CSV-file
CUSTOMERS_DF = pd.read_csv(csv_file_path)
print("CSV loaded successfully. Number of customers: ", CUSTOMERS_DF.shape[0])
print("First 5 customers: ", CUSTOMERS_DF.head(5))

CSV loaded successfully. Number of customers:  50
First 5 customers:     id          x          y
0   1  63.942680   2.501076
1   2  27.502932  22.321074
2   3  73.647121  67.669949
3   4  89.217957   8.693883
4   5  42.192182   2.979722


#### 1.2 Select random population of 14 customers

In [80]:
"""
This section selects 14 random customers from the pandas dataframe.
random_state is set to 42 to ensure reproducibility.
"""
POPULATION_SIZE = 14

# select customers randomly from customers_list_df to match needed population size.
customer_population = customers_list_df.sample(n=POPULATION_SIZE, random_state=42)
print("First 5 customers:")
print(customer_population.head(5))

# list of customer ids of selected customers
customer_ids = customer_population['id'].tolist()
print("----------------------------")
print("List of ids of selected customers:", customer_ids)

First 5 customers:
    id          x          y
13  14   9.274584   9.671638
39  40  26.488017  24.662751
30  31  98.952335  63.999976
45  46  10.964913  62.744604
17  18  37.853438  55.204063
----------------------------
List of ids of selected customers: [14, 40, 31, 46, 18, 49, 27, 26, 33, 20, 13, 5, 38, 9]


#### 1.3 Define generations, depot and vehicles.

In [81]:
# number of generations and vehicles
GENERATIONS = 3
VEHICLES = 3

# coordinated for depot
DEPOT_LOC = [0,0]

# marker of depot it chromosome list
DEPOT_MARKER = 0

# list of internal depot markers, with the constraint that all vehicles start and finish at depot (0)
num_internal_depot_markers = VEHICLES - 1

#### 1.4 Create initial random chromosome.

In [82]:
import random

"""
Constraints:
1. start at depot (0,0)
2. 3 vehicles. start and end at depot. e.g. (0,...,...,...,0,...,...,0,...,...,...,...,0)
3. each customer visited only once
"""

# create random first chromosome
def create_individual():
    """
    Creates the first random individual chromosome based on the customers string and the number of vehicles.
    """

    # create list of internal depot markers
    internal_depot_markers_list = [depot_marker] * num_internal_depot_markers

    # combine list of internal_depot_markers and customer_ids
    elements_to_shuffle = internal_depot_markers_list + customer_ids

    # shuffle all customers and internal_depot_markers randomly
    random.shuffle(elements_to_shuffle)

    # create chromosome which start at 0, and end at 0, with the customers and random point of depot-visits inbetween
    chromosome = [depot_marker] + elements_to_shuffle + [depot_marker]

    return chromosome

print("Creating individual chromosome...")
create_individual()

Creating individual chromosome...


[0, 26, 14, 0, 0, 46, 40, 9, 31, 13, 5, 20, 49, 38, 33, 27, 18, 0]

#### Create dataframe with all location variables. Depot, and customers x and y.

In [83]:
coordinates_df = pd.concat(
    [customer_population,
     pd.DataFrame([[0] + DEPOT_LOC], columns=customer_population.columns)],
)

print(coordinates_df)

    id          x          y
13  14   9.274584   9.671638
39  40  26.488017  24.662751
30  31  98.952335  63.999976
45  46  10.964913  62.744604
17  18  37.853438  55.204063
48  49  99.612138  52.911435
26  27  26.697782  93.665459
25  26  37.018097  20.950703
32  33  84.285192  77.599991
19  20  86.170690  57.735215
12  13  95.721307  33.659455
4    5  42.192182   2.979722
37  38  65.543867  39.563190
8    9  22.044062  58.926568
0    0   0.000000   0.000000


# 2. Fitness function

In [91]:
import numpy as np

def calculate_euclidean_distance(id1, id2, loc_df):
    """
    Calculates the Euclidean distance between two points.

    Based on formula: sqrt((x1 - x2)^2 + (y1 - y2)^2)

    :param id1: the id of first location
    :param id2: the id of second location
    :param loc_df: the dataframe containing the coordinates to all locations
    """

    # find x and y coordinated for id1
    x1 = loc_df.loc[loc_df['id'] == id1]['x'].iloc[0]
    y1 = loc_df.loc[loc_df['id'] == id1]['y'].iloc[0]

    # find x and y coordinates for id2
    x2 = loc_df.loc[loc_df['id'] == id2]['x'].iloc[0]
    y2 = loc_df.loc[loc_df['id'] == id2]['y'].iloc[0]

    # Euclidean calculation on objects id1 and id2
    return np.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)


def calculate_fitness(individual):
    """
    Calculate the fitness of the individual chromosome.

    :param individual:
    """
    routes_segmented = []
    routes_distances = []
    start_index_of_route = 0

    # separate all vehicle trips into individual routes (list)

    # iterate through chromosome to find depot-markers and segment into individual vehicle routes.
    for i in range(len(individual)):

        if individual[i] == depot_marker and i != 0:                    # avoid counting starting depot point
            route_segment = individual[start_index_of_route: i + 1]     # create a temporary route segment for current vehicle trip
            routes_segmented.append(route_segment)                            # append list of all routes with new route segment

            # start to look for new route from where the last ended at the depot
            start_index_of_route = i

    # calculate distances for each trip

    # collect coordinates data (x,y)

    # sum distance of all trips

    # calculate fitness based on total_trip_distance
